In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import metrics
import pickle
from sklearn.metrics import accuracy_score


In [0]:
input_set_labeled = np.genfromtxt ('training.csv', delimiter=",")
input_set=input_set_labeled[1:input_set_labeled.shape[0],:]
input_data=input_set[:,1:input_set.shape[1]]
target_data = input_set[:,0]
target_data=target_data.reshape(target_data.shape[0],1)
print(input_data.shape)
print(target_data.shape)
print(input_set.shape[1])
print(input_set.shape[0])

(21000, 784)
(21000, 1)
785
21000


In [0]:
sess = tf.Session()
init_var = tf.global_variables_initializer()
sess.run(init_var)
indices1 = tf.cast(target_data, tf.int32)
one_hot_vecs1 = tf.one_hot(indices1,np.max(sess.run(indices1))+1)
new_output=sess.run(one_hot_vecs1)
new_output=new_output.reshape(21000,10)
print(new_output)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [0]:
input_train,input_test, target_train, target_test =train_test_split(input_data,new_output,test_size=0.166667, random_state=10)
print(input_train.shape)
print(input_test.shape)
print(target_train.shape)
print(target_test.shape)

(17499, 784)
(3501, 784)
(17499, 10)
(3501, 10)


In [0]:
input_train_map=input_train.reshape(-1,28,28,1)
input_test_map=input_test.reshape(-1,28,28,1)
print(input_train_map.shape)
print(input_test_map.shape)

(17499, 28, 28, 1)
(3501, 28, 28, 1)


In [0]:

score=[]
kf = KFold(n_splits=5)
index=0
for train_index, test_index in kf.split(input_train):

    X_train, X_test_loop = input_train[train_index], input_train[test_index]
    y_train, y_test_loop = target_train[train_index], target_train[test_index]
    print(X_train.shape)
    model = DecisionTreeClassifier()
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

    filepath=str(index)+'weights.best.hdf5'
    index=index+1
    checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True,
                            mode='auto')
    callbacks_list = [checkpoint]
    model.fit(X_train, y_train)
    s=model.score(X_test_loop, y_test_loop)
    score.append(s)
    if(s>=np.max(score)):
      pkl_filename = "pickle_model.pkl"  
      with open(pkl_filename, 'wb') as file:  
        pickle.dump(model, file)
    print(model.score(X_test_loop, y_test_loop))
    

(13999, 784)
0.8202857142857143
(13999, 784)
0.8245714285714286
(13999, 784)
0.8271428571428572
(13999, 784)
0.8248571428571428
(14000, 784)
0.8153758216633323


In [0]:
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)
predict=pickle_model.predict(input_test)
print(accuracy_score(target_test,predict))

0.8189088831762353
